In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Planteamiento del Proyecto

# Representación de Datos

# Excepción personalizada
class ErrorFormatoDatosInvalido(Exception):
    """Excepción para errores en el formato de los datos de entrada."""
    def __init__(self, mensaje="Formato de datos inválido. Se esperan arrays de NumPy."):
        self.mensaje = mensaje
        super().__init__(self.mensaje)

def generar_datos_sinteticos(num_muestras=100, rango_x=(0, 10), std_ruido=1.0):
    """
    Genera datos sintéticos para regresión lineal simple: y = 4 + 3x + ε.
    """
    try:
        if not isinstance(num_muestras, int) or num_muestras <= 0:
            raise ValueError("num_muestras debe ser un entero positivo.")
        if not isinstance(rango_x, tuple) or len(rango_x) != 2 or rango_x[0] >= rango_x[1]:
            raise ValueError("rango_x debe ser una tupla (min, max) con min < max.")
        if not isinstance(std_ruido, (int, float)) or std_ruido < 0:
            raise ValueError("std_ruido debe ser un número no negativo.")

        np.random.seed(42) # Para reproducibilidad
        x = np.random.uniform(rango_x[0], rango_x[1], num_muestras).reshape(-1, 1)
        ruido = np.random.normal(0, std_ruido, num_muestras).reshape(-1, 1)
        y = 4 + 3 * x + ruido
        return x, y
    except Exception as e:
        print(f"Error al generar datos sintéticos: {e}")
        raise

# Generar datos y representarlos con NumPy
try:
    X_datos_original, y_datos = generar_datos_sinteticos(num_muestras=100)
    # Añadir columna de unos a X para el término de sesgo (intercepto)
    X_con_sesgo = np.c_[np.ones((X_datos_original.shape[0], 1)), X_datos_original]
    print("Datos Generados")
    print("X (primeros 5): ", np.round(X_datos_original[:5].flatten(), 2))
    print("y (primeros 5): ", np.round(y_datos[:5].flatten(), 2))
except ErrorFormatoDatosInvalido as e:
    print(f"Error de formato de datos: {e}")
except Exception as e:
    print(f"Error inesperado durante la generación de datos: {e}")
finally:
    print("Finalizada la sección de generación de datos.")


# Resolución del Modelo (Cálculo Cerrado - Ecuación Normal)

def resolver_ecuacion_normal(X, y):
    """
    Resuelve la regresión lineal utilizando la ecuación normal (β = (X^T X)^-1 X^T y).
    Maneja excepciones de matriz singular.
    """
    try:
        # Validar formato de datos de entrada
        if not isinstance(X, np.ndarray) or not isinstance(y, np.ndarray):
            raise ErrorFormatoDatosInvalido("Los datos de entrada para la ecuación normal deben ser arrays de NumPy.")
        if X.ndim != 2 or y.ndim != 2 or X.shape[0] != y.shape[0]:
            raise ErrorFormatoDatosInvalido("X debe ser un array 2D y 'y' un array 2D con el mismo número de filas.")

        X_transpuesta = X.T
        try:
            # Calcular (X^T X)^-1
            XtX_inversa = np.linalg.inv(X_transpuesta @ X)
        except np.linalg.LinAlgError:
            raise ValueError("La matriz (X^T X) es singular, no se puede invertir. Pruebe con más datos o elimine características linealmente dependientes.")

        # Calcular beta (parámetros)
        beta = XtX_inversa @ X_transpuesta @ y
        return beta
    except ErrorFormatoDatosInvalido as e:
        print(f"Error de formato de datos en la función 'resolver_ecuacion_normal': {e}")
        raise
    except ValueError as e:
        print(f"Error en la ecuación normal: {e}")
        raise
    except Exception as e:
        print(f"Error inesperado durante la resolución con la ecuación normal: {e}")
        raise

try:
    beta_ecuacion_normal = resolver_ecuacion_normal(X_con_sesgo, y_datos)
    print("\nParámetros del Modelo (Ecuación Normal)")
    print(f"Sesgo (theta_0): {beta_ecuacion_normal[0, 0]:.4f}")
    print(f"Pendiente (theta_1): {beta_ecuacion_normal[1, 0]:.4f}")
except Exception as e:
    print(f"No se pudo calcular con la ecuación normal debido a: {e}")
    beta_ecuacion_normal = None # Asegurar que la variable esté definida para evitar errores posteriores
finally:
    print("Finalizada la sección de ecuación normal.")


# Optimización Iterativa (Método de Descenso de Gradiente)

def calcular_costo(X, y, theta):
    """
    Calcula la función de costo (Error Cuadrático Medio - MSE).
    J(θ) = 1/(2m) * Σ(h_θ(x^(i)) - y^(i))^2
    """
    try:
        m = len(y)
        predicciones = X @ theta
        costo = (1/(2*m)) * np.sum(np.square(predicciones - y))
        return costo
    except Exception as e:
        print(f"Error al calcular el costo: {e}")
        raise

def descenso_gradiente(X, y, tasa_aprendizaje=0.01, num_iteraciones=1500):
    """
    Implementa el algoritmo de Descenso de Gradiente para regresión lineal.
    """
    try:
        # Validar formato de datos de entrada
        if not isinstance(X, np.ndarray) or not isinstance(y, np.ndarray):
            raise ErrorFormatoDatosInvalido("Los datos de entrada para descenso de gradiente deben ser arrays de NumPy.")
        if X.ndim != 2 or y.ndim != 2 or X.shape[0] != y.shape[0]:
            raise ErrorFormatoDatosInvalido("X debe ser un array 2D y 'y' un array 2D con el mismo número de filas.")
        if not isinstance(tasa_aprendizaje, (int, float)) or tasa_aprendizaje <= 0:
            raise ValueError("tasa_aprendizaje debe ser un número positivo.")
        if not isinstance(num_iteraciones, int) or num_iteraciones <= 0:
            raise ValueError("num_iteraciones debe ser un entero positivo.")

        m = len(y)
        theta = np.zeros((X.shape[1], 1)) # Inicializar parámetros (sesgo y pendiente) con ceros
        historial_costo = []

        for i in range(num_iteraciones):
            predicciones = X @ theta
            errores = predicciones - y
            # Gradiente: (1/m) * X^T * errores
            gradiente = (1/m) * X.T @ errores
            theta = theta - tasa_aprendizaje * gradiente
            historial_costo.append(calcular_costo(X, y, theta))
        return theta, historial_costo
    except ErrorFormatoDatosInvalido as e:
        print(f"Error de formato de datos en la función 'descenso_gradiente': {e}")
        raise
    except ValueError as e:
        print(f"Error en descenso de gradiente: {e}")
        raise
    except Exception as e:
        print(f"Error inesperado durante la ejecución de descenso de gradiente: {e}")
        raise

try:
    theta_descenso_gradiente, historial_costo_descenso_gradiente = descenso_gradiente(X_con_sesgo, y_datos, tasa_aprendizaje=0.01, num_iteraciones=1500)
    print("\nParámetros del Modelo (Descenso de Gradiente)")
    print(f"Sesgo (theta_0): {theta_descenso_gradiente[0, 0]:.4f}")
    print(f"Pendiente (theta_1): {theta_descenso_gradiente[1, 0]:.4f}")

    if beta_ecuacion_normal is not None:
        costo_final_ecuacion_normal = calcular_costo(X_con_sesgo, y_datos, beta_ecuacion_normal)
        print(f"Costo final (Ecuación Normal): {costo_final_ecuacion_normal:.4f}")
    else:
        costo_final_ecuacion_normal = "No calculado"
        print("Costo final (Ecuación Normal): No disponible debido a un error previo.")

    costo_final_descenso_gradiente = historial_costo_descenso_gradiente[-1]
    print(f"Costo final (Descenso de Gradiente): {costo_final_descenso_gradiente:.4f}")

except Exception as e:
    print(f"No se pudo calcular con descenso de gradiente debido a: {e}")
    theta_descenso_gradiente = None # Asegurar que la variable esté definida
    historial_costo_descenso_gradiente = []
finally:
    print("Finalizada la sección de descenso de gradiente.")

# 2. Visualización y Análisis

# Graficar el conjunto de datos junto con la recta de regresión
if theta_descenso_gradiente is not None and beta_ecuacion_normal is not None:
    plt.figure(figsize=(10, 6))
    plt.scatter(X_datos_original, y_datos, label='Datos Sintéticos', alpha=0.6)

    # Línea de regresión de Ecuación Normal
    x_linea = np.array([X_datos_original.min(), X_datos_original.max()]).reshape(-1, 1)
    X_linea_con_sesgo = np.c_[np.ones((x_linea.shape[0], 1)), x_linea]
    y_pred_ecuacion_normal = X_linea_con_sesgo @ beta_ecuacion_normal
    plt.plot(x_linea, y_pred_ecuacion_normal, color='red', label=f'Regresión (Ecuación Normal): y = {beta_ecuacion_normal[0,0]:.2f} + {beta_ecuacion_normal[1,0]:.2f}x')

    # Línea de regresión de Descenso de Gradiente
    y_pred_descenso_gradiente = X_linea_con_sesgo @ theta_descenso_gradiente
    plt.plot(x_linea, y_pred_descenso_gradiente, color='green', linestyle='--', label=f'Regresión (Descenso de Gradiente): y = {theta_descenso_gradiente[0,0]:.2f} + {theta_descenso_gradiente[1,0]:.2f}x')

    plt.xlabel('Variable Independiente (x)')
    plt.ylabel('Variable Dependiente (y)')
    plt.title('Regresión Lineal Simple: Comparación de Métodos')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("\nNo se pueden generar gráficos de regresión porque uno o ambos métodos fallaron.")

# Generar un gráfico que muestre la evolución de la función de costo
if historial_costo_descenso_gradiente:
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(historial_costo_descenso_gradiente)), historial_costo_descenso_gradiente, color='blue')
    plt.xlabel('Número de Iteraciones')
    plt.ylabel('Función de Costo (MSE)')
    plt.title('Evolución de la Función de Costo en Descenso de Gradiente (GD)')
    plt.grid(True)
    plt.show()
else:
    print("\nNo se puede generar el gráfico de evolución del costo porque el descenso de gradiente falló o no se ejecutó.")
